### 모델 라이브러리

In [57]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

In [59]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveLGBM(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 50, 500)
        , 'max_depth': trial.suggest_int('max_depth', 2, 20)
        , 'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1)
        , 'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        
        , 'random_state': 0
        , 'verbose' : -1
    }
    
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"].astype(int),
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveLGBM(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-22 17:27:04,431] A new study created in memory with name: no-name-488f5716-7e52-4e05-b8d2-8f78b80776da


[I 2024-02-22 17:27:13,440] Trial 0 finished with value: 0.8276249298147108 and parameters: {'num_leaves': 297, 'max_depth': 15, 'n_estimators': 643, 'learning_rate': 0.05494343511669279, 'min_child_samples': 45}. Best is trial 0 with value: 0.8276249298147108.
[I 2024-02-22 17:27:20,787] Trial 1 finished with value: 0.8280467445742904 and parameters: {'num_leaves': 341, 'max_depth': 10, 'n_estimators': 903, 'learning_rate': 0.09640261328960191, 'min_child_samples': 41}. Best is trial 1 with value: 0.8280467445742904.
[I 2024-02-22 17:27:30,052] Trial 2 finished with value: 0.8333333333333333 and parameters: {'num_leaves': 407, 'max_depth': 12, 'n_estimators': 611, 'learning_rate': 0.09263406719097345, 'min_child_samples': 11}. Best is trial 2 with value: 0.8333333333333333.
[I 2024-02-22 17:27:31,243] Trial 3 finished with value: 0.8018757327080891 and parameters: {'num_leaves': 89, 'max_depth': 2, 'n_estimators': 850, 'learning_rate': 0.0780375183440352, 'min_child_samples': 88}. Bes

Best trial: score 0.844170403587444, 
params {'num_leaves': 361, 'max_depth': 7, 'n_estimators': 722, 'learning_rate': 0.06729510233730235, 'min_child_samples': 32}


### 모델 성능확인

In [63]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [60]:
# LightGBM
model = LGBMClassifier(
	n_estimators=722
    , num_leaves=361
    , max_depth=7
    , learning_rate=0.06729510233730235
    , min_child_samples=32
    , verbose=-1
	, random_state=0
)

### 모델 학습

In [61]:
model.fit(x_train, y_train)

LGBMClassifier(learning_rate=0.06729510233730235, max_depth=7,
               min_child_samples=32, n_estimators=722, num_leaves=361,
               random_state=0, verbose=-1)

In [64]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,753,194
False,84,10829


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.97656,0.899642,0.795143,0.84417,0.975896


In [65]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

750

### 제출파일 생성

In [828]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_lgbm_optuna.csv", index=False)

.